文本分类的目的是将文本文档分为不同的类。     
采用一种技术，它基于一种叫做tf-idf的统计数据，它表示词频-逆文档频率(term frequency-inverse document frequency)     
这个统计工具有助于理解一个单词在一组文档中对某一个文档的重要性

**词频(The term frequency,TF)** 表示一个单词在给定文档中出现的频次。由于不同文档的长度不同，
这些频次的直方图看起来会相差很大，因此需要将其规范化，使这些频次可以在平等的环境下对比。
为了实现规范化，我们用频次除以文档中所有单词的个数。        
**逆文档频率(inverse document frequency,IDF)** 表示给定单词的重要性。当需要计算词频时，假定所有单词是
同等重要的。为了抗衡那些经常出现的单词的频率，如is,be等，需要一个系数将其权重变小。
我们需要计算文档总数目除以该单词出现的文档数目的比值。逆文档频率对该比值取对数值。

In [2]:
from sklearn.datasets import fetch_20newsgroups

# 选择一个类型列表，并且用词典映射的方式定义。这些类型是加载的新闻组的数据集中的一部分
category_map = {'misc.forsale': 'Sales', 'rec.motorcycles': 'Motorcycles', 
        'rec.sport.baseball': 'Baseball', 'sci.crypt': 'Cryptography', 
        'sci.space': 'Space'}

# 基于刚刚定义的类型加载训练数据
training_data = fetch_20newsgroups(subset='train', 
        categories=category_map.keys(), shuffle=True, random_state=7)

# 导入特征提取器
from sklearn.feature_extraction.text import CountVectorizer

# 用训练数据提取特征
vectorizer = CountVectorizer()
X_train_termcounts = vectorizer.fit_transform(training_data.data)
print("\nDimensions of training data:", X_train_termcounts.shape)

# 训练分类器
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
# 定义一些随机输入的句子
input_data = [
    "The curveballs of right handed pitchers tend to curve to the left", 
    "Caesar cipher is an ancient form of encryption",
    "This two-wheeler is really good on slippery roads"
]
# 定义tf-idf变换器对象并训练
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_termcounts)
# 得到特征向量后，用该数据训练多项式朴素贝叶斯分类器
classifier = MultinomialNB().fit(X_train_tfidf, training_data.target)
# 用词频统计转换输入数据
X_input_termcounts = vectorizer.transform(input_data)
# 用tf-idf变换器交换输入数据
X_input_tfidf = tfidf_transformer.transform(X_input_termcounts)

# 用训练过的分类器预测这些输入句子的输出类型
# 预测输出类型
predicted_categories = classifier.predict(X_input_tfidf)
# 打印输出结果
for sentence, category in zip(input_data, predicted_categories):
    print('\nInput:', sentence, '\nPredicted category:', \
            category_map[training_data.target_names[category]])


Dimensions of training data: (2968, 40605)

Input: The curveballs of right handed pitchers tend to curve to the left 
Predicted category: Baseball

Input: Caesar cipher is an ancient form of encryption 
Predicted category: Cryptography

Input: This two-wheeler is really good on slippery roads 
Predicted category: Motorcycles
